In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound as rb
from os.path import isfile
from sympy import init_printing
init_printing()

In [2]:
from celmech.secular import LaplaceLagrangeSystem

In [72]:
def eccMatrix(alpha, mass):
    mu1, mu2, mu3 = mass
    P1, P2, P3 = alpha**(3/2), 1, 1/alpha**(3/2)

    sim = rb.Simulation()
    sim.add(m=1.)
    sim.add(m=mu1, P=P1)
    sim.add(m=mu2, P=P2)
    sim.add(m=mu3, P=P3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = sim.particles[1].P/20
    sim.collision = "direct"
    align_simulation(sim)

    lsys = LaplaceLagrangeSystem.from_Simulation(sim)
#     eigvals = lsys.eccentricity_eigenvalues()
#     Tsec = np.max(np.abs(2 * np.pi / eigvals))
#     sim.automateSimulationArchive('sa_laplace_lagrange.bin',interval=Tsec/2048,deletefile=True)
#     sim.integrate(0.05*3e5)
#     sa = rb.SimulationArchive('sa_laplace_lagrange.bin')
    
#     nb_results = get_simarchive_integration_results(sa)
#     sim = sa[0]
#     lsys = LaplaceLagrangeSystem.from_Simulation(sim)
#     sec_soln = lsys.secular_solution(nb_results['time'])
    
    M = lsys.Neccentricity_matrix
    print(f"eccentricity matrix =")
    print(M)
    
    return M

In [73]:
def eccMatrix_rotated(alpha, mass):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    
    R = np.array([[np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)],
             [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
             [np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)]])
    M = eccMatrix(alpha, mass)
    
    return np.matmul(np.matmul(R, M), np.transpose(R))

In [75]:
eccMatrix_rotated(0.995, (1e-7, 1e-7, 1e-7))

eccentricity matrix =
[[-0.00499597  0.00399314  0.00099391]
 [ 0.00399314 -0.00795785  0.00396323]
 [ 0.00099391  0.00396323 -0.00495109]]


array([[-1.44951531e-06,  6.10939182e-06,  2.24915354e-08],
       [ 6.10939182e-06, -5.96743828e-03,  3.02258006e-05],
       [ 2.24915354e-08,  3.02258006e-05, -1.19360129e-02]])

In [76]:
# Two-body case

def test(alpha, mass):
    mu1, mu2 = mass
    P1, P2 = alpha**(3/2), 1

    sim = rb.Simulation()
    sim.units = ('yr', 'AU', 'Msun')
    sim.add(m=1.)
    sim.add(m=mu1, P=P1)
    sim.add(m=mu2, P=P2)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = sim.particles[1].P/20
    sim.collision = "direct"
    align_simulation(sim)
    
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)
    M = lsys.Neccentricity_matrix
    print(f"eccentricity matrix =")
    print(M)

In [77]:
test(0.995, (1e-4, 1e-4))

eccentricity matrix =
[[-3.26933276  3.26406044]
 [ 3.26406044 -3.26028217]]
